In [1]:
# convenience directory variables
TRAIN_DIR = '/kaggle/input/rsna-2023-atd-reduced-256-5mm/reduced_256_tickness_5'

In [2]:
import pandas as pd
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
import shutil

In [3]:
train = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv")
train.tail()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
3142,9951,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3143,9960,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3145,9980,1,0,1,0,1,0,0,1,0,0,0,0,1,1
3146,9983,1,0,1,0,1,0,0,1,0,0,0,0,1,1


In [4]:
train_series_meta = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv")
print("Number of unique patients in train dataset: ", len(train_series_meta['patient_id'].unique()))
print("Number of samples in train dataset: ", len(train_series_meta['patient_id']))
train_series_meta.head(100)

Number of unique patients in train dataset:  3147
Number of samples in train dataset:  4711


,patient_id,series_id,aortic_hu,incomplete_organ
0,10004,21057,146.00,0
1,10004,51033,454.75,0
2,10005,18667,187.00,0
3,10007,47578,329.00,0
4,10026,29700,327.00,0
...,...,...,...,...
95,11312,51300,179.00,0
96,11313,48992,297.00,0
97,11335,24276,111.00,0
98,11335,39434,216.00,0


In [5]:
image_level_labels = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/image_level_labels.csv")
print("Unique series: ", len(image_level_labels['series_id'].unique()))
image_level_labels.head(1000)

Unique series:  330


,patient_id,series_id,instance_number,injury_name
0,10004,21057,362,Active_Extravasation
1,10004,21057,363,Active_Extravasation
2,10004,21057,364,Active_Extravasation
3,10004,21057,365,Active_Extravasation
4,10004,21057,366,Active_Extravasation
...,...,...,...,...
995,12332,15276,60,Bowel
996,12332,15276,61,Bowel
997,12332,15276,62,Bowel
998,12332,15276,63,Bowel


In [6]:
train_d = train.loc[(train['kidney_healthy']==1) & (train['liver_healthy']==1) & (train['spleen_healthy']==1) & (train['any_injury']==0)]
train_d = train_series_meta.merge(train_d, on='patient_id', how='right')
train_d = train_d.loc[train_d['incomplete_organ']==0]

In [7]:
paths = []
for _, row in tqdm(train_d.iterrows(), total = len(train_d)):
    path = f"{TRAIN_DIR}/{int(row['patient_id'])}/{int(row['series_id'])}"
    images = os.listdir(path)
    filenames = [int(filename.split('.')[0]) for filename in images]
    images = list(map(lambda x: str(x)+'.jpeg', sorted(filenames)))
    [paths.append(f"{path}/{image}") for image in images]


  0%|          | 0/3214 [00:00<?, ?it/s]

In [8]:
len(paths)

335180

In [9]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import random
import numpy as np
from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
import os
import glob
import torch
from PIL import Image

class DiffDataset(torch.utils.data.Dataset):

    def __init__(self, paths:list[str], transforms = None, limit:int = None, normalize=True):
        
        self.limit = limit
        self.transforms = transforms
        self.normalize = normalize
        self.image_paths = paths
        if limit:
            self.image_paths = self.image_paths[0:limit]
    
    def __getitem__(self, index: int)->tuple[np.ndarray, np.ndarray]:
        img = np.array(Image.open(self.image_paths[index]), dtype = np.float32)
        
        if self.normalize:
            img = img/255.0
            
        if self.transforms is not None:
            augmented = self.transforms(image=img)
            img = augmented['image']
            img.unsqueeze(0)
        return img
    
    def __len__(self):
        return(len(self.image_paths))

# Model

In [11]:
!pip install denoising_diffusion_pytorch
!pip install git+https://github.com/huggingface/accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-4gkgwchs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-4gkgwchs
  Resolved https://github.com/huggingface/accelerate to commit e2ae254008061b3e53fc1c97f88d65743a857e75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-none-any.whl size=255448 sha256=7bc5c28fa1af0f6eaad7614624297e04b078227563f1597f6a33c07c0f4e446a
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zp_9w7c/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: acce

In [12]:
from accelerate import Accelerator, notebook_launcher
from accelerate.utils import set_seed

from denoising_diffusion_pytorch import Unet, ElucidatedDiffusion
from ema_pytorch import EMA

from torch.optim import Adam
from torch.utils.data import DataLoader, Sampler

from multiprocessing import cpu_count
from pathlib import Path

import torch.nn as nn


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [13]:
def construct_dataset(paths, transforms = None, limit: int = None, normalize = True):    
    # For training a simple RandomResizedCrop will be used
    assert len(paths) > 1000, "Dataset must have more than 1000 images."
    if not transforms:
        transforms = A.Compose([
            ToTensorV2()
        ])
    
    return DiffDataset(paths, transforms = transforms, limit = limit, normalize = normalize)

In [22]:
def construct_network():
    unet = Unet(dim=64, dim_mults=(1, 2, 4, 8), channels=1, learned_sinusoidal_dim = 16, 
                learned_sinusoidal_cond = True, random_fourier_features = False, attn_dim_head = 16,
               resnet_block_groups = 2, attn_heads = 1, learned_variance = True)
    model = ElucidatedDiffusion(
        unet,
        image_size=256,
        channels = 1,
        num_sample_steps = 34, # number of sampling steps
        sigma_min = 0.002,     # min noise level
        sigma_max = 40,        # max noise level
        sigma_data = 0.5,      # standard deviation of data distribution
        rho = 7,               # controls the sampling schedule
        P_mean = -1.2,         # mean of log-normal distribution from which noise is drawn for training
        P_std = 1.2,           # standard deviation of log-normal distribution from which noise is drawn for training
        S_churn = 80,          # parameters for stochastic sampling - depends on dataset, Table 5 in apper
        S_tmin = 0.05,
        S_tmax = 50,
        S_noise = 1.003,
    )
    return model

In [15]:
#a = construct_dataset(paths)
#t_im = a[1000]#.squeeze().numpy()
#t_im.shape
#plt.imshow(t_im)

In [16]:
#net = construct_network()

In [17]:
#from einops import rearrange, repeat, reduce

def normalize_to_neg_one_to_one(img):
    return img * 2 - 1

def ff(net, images):
    batch_size, c, h, w, device, image_size, channels = *images.shape, images.device, net.image_size, net.channels

    assert h == image_size and w == image_size, f'height and width of image must be {image_size}'
    assert c == channels, 'mismatch of image channels'

    images = normalize_to_neg_one_to_one(images)

    sigmas = net.noise_distribution(batch_size)
    padded_sigmas = rearrange(sigmas, 'b -> b 1 1 1')

    noise = torch.randn_like(images)

    noised_images = images + padded_sigmas * noise  # alphas are 1. in the paper
    
    self_cond = None
    denoised = net.preconditioned_network_forward(noised_images, sigmas, self_cond)
    return noised_images, denoised

In [18]:
#noised, denoised = ff(net, a[1000].unsqueeze(0))

In [19]:
#plt.imshow(noised.squeeze().detach().numpy())

In [20]:
def get_rank():
    return torch.distributed.get_rank() if torch.distributed.is_initialized() else 0


def get_world_size():
    return torch.distributed.get_world_size() if torch.distributed.is_initialized() else 1

def exists(x):
    return x is not None

class InfiniteSampler(torch.utils.data.Sampler):
    def __init__(self, dataset, rank=0, num_replicas=1, shuffle=True, seed=0, window_size=0.5):
        assert len(dataset) > 0
        assert num_replicas > 0
        assert 0 <= rank < num_replicas
        assert 0 <= window_size <= 1
        super().__init__(dataset)
        self.dataset = dataset
        self.rank = rank
        self.num_replicas = num_replicas
        self.shuffle = shuffle
        self.seed = seed
        self.window_size = window_size

    def __iter__(self):
        order = np.arange(len(self.dataset))
        rnd = None
        window = 0
        if self.shuffle:
            rnd = np.random.RandomState(self.seed)
            rnd.shuffle(order)
            window = int(np.rint(order.size * self.window_size))

        idx = 0
        while True:
            i = idx % order.size
            if idx % self.num_replicas == self.rank:
                yield order[i]
            if window >= 2:
                j = (i - rnd.randint(window)) % order.size
                order[i], order[j] = order[j], order[i]
            idx += 1


In [24]:
import os
import time
import copy
import json
import pickle
import psutil
import numpy as np
import torch

#----------------------------------------------------------------------------

def train(
    run_dir             = './results',      # Output directory.
    dataset_options     = {'paths':paths, 'transforms': None, 'limit': None, 'normalize': True},       # Option for dataset creator 
    network_kwargs      = {},       # Options for model.
    optimizer_kwargs    = {'lr': 1e-4, 'betas': (0.9, 0.99)},       # Options for optimizer.
    ema_kwargs          = {'ema_beta': 0.995, 'ema_halflife_kimg': 500, 'ema_rampup_ratio': 0.05},
    split_batches       = True,     # Split batches?
    mixes_pecision      = 'fp16',
    seed                = 12,        # Global random seed.
    batch_size:int      = 240,      # Total batch size for one training iteration.
    total_kimg          = 50000,   # Training duration, measured in thousands of training images.
    #ema_halflife_kimg   Half-life of the exponential moving average (EMA) of model weights.
    #ema_rampup_ratio    EMA ramp-up coefficient, None = no rampup.
    lr_rampup_kimg      = 10000,    # Learning rate ramp-up duration.
    loss_scaling        = 1,        # Loss scaling factor for reducing FP16 under/overflows.
    snapshot_ticks      = 100,       # How often to save network snapshots, None = disable.
    resume_pkl          = '/kaggle/working/results/model-save.pt',     # Start from the given network snapshot, None = random initialization.
    resume_kimg         = 0,        # Start from the given training progress.
    batch_gpu           = 8,      # Limit batch size per GPU, None = no limit.
    max_training_time   = 6      # Maximum training time, in hours.
):

    # Initialize accelerator.
    accelerator = Accelerator(
        split_batches = split_batches,
        mixed_precision = 'fp16'
    )
    device = accelerator.device
    
    if accelerator.is_main_process:
        results_folder = Path(run_dir)
        results_folder.mkdir(exist_ok = True)

    # Select batch size per GPU.
    accelerator.print("Splitting batches...")
    batch_gpu_total = batch_size // get_world_size() #6
    if batch_gpu is None or batch_gpu > batch_gpu_total:
        batch_gpu = batch_gpu_total
    num_accumulation_rounds = batch_gpu_total // batch_gpu
    #accelerator.print(batch_gpu, num_accumulation_rounds)
    assert batch_size == batch_gpu * num_accumulation_rounds * get_world_size()
    accelerator.print(f"batch_size: {batch_gpu}, num_accumulation_rounds: {num_accumulation_rounds}, batch_gpu_total: {batch_gpu_total}")

    # Load dataset.
    accelerator.print('Loading dataset...')
    dataset_obj = construct_dataset(**dataset_options)
    dataset_sampler = InfiniteSampler(dataset=dataset_obj, rank=get_rank(), num_replicas=get_world_size(), seed=seed)
    dataset_iterator = iter(torch.utils.data.DataLoader(dataset=dataset_obj, sampler=dataset_sampler, batch_size=batch_gpu))

    # Construct network.
    accelerator.print('Constructing network...')
    net = construct_network()
    
    # Setup optimizer.
    accelerator.print('Setting up optimizer...')
    optimizer =  Adam(net.parameters(), **optimizer_kwargs)
    
    # Accelerator prepare
    net, optimizer, dataset_iterator = accelerator.prepare(
        net, optimizer, dataset_iterator
    )
    
    
    if accelerator.is_main_process:
        ema_beta = ema_kwargs['ema_beta']
        ema_halflife_kimg = ema_kwargs['ema_halflife_kimg']
        ema_rampup_ratio=ema_kwargs['ema_rampup_ratio']
        ema = copy.deepcopy(net).eval().requires_grad_(False)
        #ema.to(device)

    # Resume training from previous snapshot.
    if resume_pkl is not None:
        data = torch.load(resume_pkl, map_location=device)
        m = accelerator.unwrap_model(net)
        net.load_state_dict(data['model'])
        resume_kimg = data['kimg']
        optimizer.load_state_dict(data['opt'])
        if accelerator.is_main_process:
            ema.load_state_dict(data["ema"])
        if 'version' in data:
            print(f"loading from version {data['version']}")
        if exists(accelerator.scaler) and exists(data['scaler']):
            accelerator.scaler.load_state_dict(data['scaler'])
        del data # conserve memory
        
        net, optimizer, dataset_iterator = accelerator.prepare(
            net, optimizer, dataset_iterator
    )
        
    
    # Train.
    accelerator.print(f'Training for {total_kimg} kimg...')
    accelerator.print()
    cur_nimg = resume_kimg * 1000
    cur_tick = 0
    start_time = time.time()
    time_limit = max_training_time*3600
    
    with tqdm(initial = cur_nimg, total = total_kimg*1000, disable = not accelerator.is_main_process) as pbar:
        while True:
            total_loss = 0.
            # Accumulate gradients.
            optimizer.zero_grad(set_to_none=True)
            for round_idx in range(num_accumulation_rounds):
                images = next(dataset_iterator)
                with accelerator.autocast():
                    #accelerator.print(images.shape)
                    loss = net(images)
                    loss = loss / num_accumulation_rounds
                    total_loss += loss.item()
                accelerator.backward(loss)
            pbar.set_description(f'loss: {total_loss:.4f}')
            accelerator.wait_for_everyone()

            # Update weights.
            for g in optimizer.param_groups:
                g['lr'] = optimizer_kwargs['lr'] * min(cur_nimg / max(lr_rampup_kimg * 1000, 1e-8), 1)
            for param in net.parameters():
                if param.grad is not None:
                    torch.nan_to_num(param.grad, nan=0, posinf=1e5, neginf=-1e5, out=param.grad)
            optimizer.step()

            # Update EMA.
            if accelerator.is_local_main_process:
                ema_halflife_nimg = ema_halflife_kimg * 1000
                if ema_rampup_ratio is not None:
                    ema_halflife_nimg = min(ema_halflife_nimg, cur_nimg * ema_rampup_ratio)
                ema_beta = 0.5 ** (batch_size / max(ema_halflife_nimg, 1e-8))
                for p_ema, p_net in zip(ema.parameters(), net.parameters()):
                    p_ema.copy_(p_net.detach().lerp(p_ema, ema_beta))

            cur_nimg += batch_size
            done = (cur_nimg >= total_kimg * 1000) or (time.time() - start_time > time_limit)

            # Save network snapshot.
            if (snapshot_ticks is not None) and (done or cur_tick % snapshot_ticks == 0):
                    
                if accelerator.is_local_main_process:
                    data = {
                        'kimg': cur_nimg//1000,
                        'model': accelerator.get_state_dict(net),
                        'opt': optimizer.state_dict(),
                        'ema': ema.state_dict(),
                        'scaler': accelerator.scaler.state_dict() if exists(accelerator.scaler) else None,
                    }
                    torch.save(data, str(os.path.join(run_dir, f'model-save2.pt')))
                    del data # conserve memory

            # Update state.
            cur_tick += 1
            pbar.update(batch_size)
            if done:
                break

    # Done.
    accelerator.print()
    accelerator.print('Exiting...')

#----------------------------------------------------------------------------

In [25]:
notebook_launcher(train, (), num_processes=2)

Launching training on 2 GPUs.
Splitting batches...
batch_size: 8, num_accumulation_rounds: 15, batch_gpu_total: 120
Loading dataset...
Constructing network...
Setting up optimizer...
Training for 50000 kimg...



  1%|          | 336000/50000000 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/denoising_diffusion_pytorch/elucidated_diffusion.py:272: UserWarning: Using a target size (torch.Size([8, 1, 256, 256])) that is different to the input size (torch.Size([8, 2, 256, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  losses = F.mse_loss(denoised, images, reduction = 'none')
/opt/conda/lib/python3.10/site-packages/denoising_diffusion_pytorch/elucidated_diffusion.py:272: UserWarning: Using a target size (torch.Size([8, 1, 256, 256])) that is different to the input size (torch.Size([8, 2, 256, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  losses = F.mse_loss(denoised, images, reduction = 'none')
/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: Grad strides do not match bucket view strides. This may indicate grad was not created according to the gradient layout contract, or

KeyboardInterrupt: 

!pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

def get_Unet():
    model =  smp.Unet(
                 encoder_name='resnet34',
                 encoder_weights='imagenet',
                 in_channels=1
                 )
    return model

import os
import time
import copy
import json
import pickle
import psutil
import numpy as np
import torch

#----------------------------------------------------------------------------

def train(
    run_dir             = './results',      # Output directory.
    dataset_options     = {'paths':paths, 'transforms': None, 'limit': 25, 'normalize': True},       # Option for dataset creator 
    network_kwargs      = {},       # Options for model.
    epochs              = 10,
    optimizer_kwargs    = {'lr': 1e-4, 'betas': (0.9, 0.99)},       # Options for optimizer.
    split_batches       = True,     # Split batches?
    mixes_pecision      = 'fp16',
    seed                = 12,        # Global random seed.
    lr_rampup_kimg      = 10000,    # Learning rate ramp-up duration.
    snapshot_ticks      = 2,       # How often to save network snapshots, None = disable.
    batch_gpu           = 6      # Limit batch size per GPU, None = no limit.
):

    # Initialize accelerator.
    accelerator = Accelerator(
        split_batches = split_batches,
        mixed_precision = 'fp16'
    )
    device = accelerator.device
    
    if accelerator.is_main_process:
        results_folder = Path(run_dir)
        results_folder.mkdir(exist_ok = True)

    # Select batch size per GPU.
    accelerator.print("Splitting batches...")
    accelerator.print(f"batch_size: {batch_gpu}")
    
    #mse = nn.MSELoss(reduction='sum').to(accelerator.device)
    # Load dataset.
    accelerator.print('Loading dataset...')
    dataset_obj = construct_dataset(**dataset_options)
    #dataset_sampler = InfiniteSampler(dataset=dataset_obj, rank=get_rank(), num_replicas=get_world_size(), seed=seed)
    dataset_iterator = torch.utils.data.DataLoader(dataset=dataset_obj, batch_size=batch_gpu)

    # Construct network.
    accelerator.print('Constructing network...')
    net = get_Unet()
    
    # Setup optimizer.
    accelerator.print('Setting up optimizer...')
    optimizer =  Adam(net.parameters(), **optimizer_kwargs)
    
    # Accelerator prepare
    net, optimizer, dataset_iterator = accelerator.prepare(
        net, optimizer, dataset_iterator
    )

    # Train.
    accelerator.print(f'Training for {epochs} epochs...')
    accelerator.print()
    epoch = 0
    while epoch<epochs:
        
        accelerator.print(f"Epoch {epoch+1}/{epochs}")
        net.train()
        with tqdm(initial = 0, total = len(dataset_obj), disable = not accelerator.is_main_process) as pbar:
            for images in dataset_iterator:
                output = net(images)
                loss = torch.nn.functional.mse_loss(output, images, reduction = 'sum')
                
                #accelerator.wait_for_everyone()
                accelerator.backward(loss)
                      
                optimizer.step()
                optimizer.zero_grad()
                pbar.set_description(f'loss: {loss:.4f}')
                pbar.update(batch_gpu)
            
            # Save network snapshot.
            if (snapshot_ticks is not None) and (epoch % snapshot_ticks == 0):
                if accelerator.is_local_main_process:
                    with torch.inference_mode():
                        fig, ax = plt.subplots(2, output.shape[0])
                        for p in range(output.shape[0]):
                            ax[0, p].imshow(images[p,:,:].cpu().detach().squeeze().numpy()*255.0)
                            ax[1, p].imshow(output[p,:,:].cpu().detach().squeeze().numpy()*255.0)
                        plt.show()
                    data = {
                        'epoch': epoch,
                        'model': accelerator.get_state_dict(net),
                        'opt': optimizer.state_dict(),
                        'scaler': accelerator.scaler.state_dict() if exists(accelerator.scaler) else None,
                    }
                    torch.save(data, str(os.path.join(run_dir, f'model-{epoch}.pt')))
                    del data # conserve memory

        if accelerator.is_local_main_process:
            epoch += 1
    # Done.
    accelerator.print()
    accelerator.print('Exiting...')

#----------------------------------------------------------------------------

notebook_launcher(train, (), num_processes=2)